# Modelo **encoder-decoder** para resolver sumas


In [2]:
# NumPy: operaciones numéricas básicas, creación de arrays y manejo de datos
import numpy as np

# Keras Model API: para definir modelos funcionales (encoder-decoder en este caso)
from tensorflow.keras.models import Model

# Capas de red neuronal:
# - Input: define la forma de los datos de entrada
# - LSTM: capa recurrente con memoria de largo plazo, usada tanto en encoder como decoder
# - Dense: capa totalmente conectada, usada para proyectar a la salida (softmax sobre vocabulario)
from tensorflow.keras.layers import Input, LSTM, Dense

# to_categorical: convierte índices enteros a vectores one-hot (no usado directamente si ya vectorizamos manualmente)
from tensorflow.keras.utils import to_categorical

# random: para generar números aleatorios, usado en la creación del dataset sintético
import random



In [3]:

# Parámetros
DIGITS = 3 # Número máximo de dígitos a considerar en la Suma
TRAINING_SIZE = 100000  # Es un 10% del total de números posibles
TEST_SIZE = 20000 # Es un 2% del total
CHARS = '0123456789+ '
MAXLEN = DIGITS + 1 + DIGITS  # '345+678' -> 7 caracteres
ANSWER_LEN = DIGITS + 1       # Hasta 1998 → 4 caracteres

In [4]:
# Mapeos char ↔ int
char_to_index = {c: i for i, c in enumerate(CHARS)}
index_to_char = {i: c for c, i in char_to_index.items()}
NUM_CHARS = len(CHARS)

In [5]:
def generate_data(size):
    questions = []  # Lista de entradas tipo string, por ejemplo: '123+45'
    answers = []    # Lista de salidas (targets), por ejemplo: '168'

    for _ in range(size):
        # Generar dos números aleatorios de hasta DIGITS dígitos
        a = random.randint(1, 10**DIGITS - 1)
        b = random.randint(1, 10**DIGITS - 1)

        # Crear string de la forma 'a+b', alineado a la derecha para que todas las entradas tengan la misma longitud
        q = f'{a}+{b}'.ljust(MAXLEN)

        # Calcular la suma y convertirla en string, también alineada a la derecha
        a_str =  str(a + b).ljust(ANSWER_LEN)

        # Agregar a las listas
        questions.append(q)
        answers.append(a_str)

    return questions, answers  # Devuelve listas paralelas de strings: inputs y outputs


In [6]:
generate_data(2)

(['503+914', '318+75 '], ['1417', '393 '])

In [7]:
# One-hot encoding
def vectorize(seqs, maxlen):
    # Inicializa un array 3D para representar las secuencias como vectores one-hot
    # Forma: (número de secuencias, longitud máxima, tamaño del vocabulario)
    x = np.zeros((len(seqs), maxlen, NUM_CHARS), dtype=np.float32)

    # Recorre cada secuencia y cada carácter
    for i, seq in enumerate(seqs):
        for t, char in enumerate(seq):
            # Marca con un 1 la posición correspondiente al carácter actual
            x[i, t, char_to_index[char]] = 1

    return x  # Devuelve el array codificado one-hot


In [8]:
vectorize( ['123+45 '], MAXLEN)

array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]], dtype=float32)

In [9]:
# Datos
questions, answers = generate_data(TRAINING_SIZE + TEST_SIZE)
x_train = vectorize(questions[:TRAINING_SIZE], MAXLEN)
y_train = vectorize(answers[:TRAINING_SIZE], ANSWER_LEN)
x_test = vectorize(questions[TRAINING_SIZE:], MAXLEN)
y_test = vectorize(answers[TRAINING_SIZE:], ANSWER_LEN)


## Modelo encoder - decoder (Tamaño fijo)


In [10]:
# Modelo encoder-decoder
# Modelo encoder-decoder

HIDDEN_SIZE = 300  # Tamaño del estado oculto de las LSTM (capacidad de memoria del modelo)

# Entrada del codificador (encoder): una secuencia de longitud MAXLEN codificada one-hot con NUM_CHARS posibles caracteres
encoder_inputs = Input(shape=(MAXLEN, NUM_CHARS))

# LSTM del encoder: procesa toda la secuencia y devuelve el último estado oculto y de celda
encoder = LSTM(HIDDEN_SIZE, return_state=True)
_, state_h, state_c = encoder(encoder_inputs)  # Solo nos interesan los estados finales, no la salida completa

# Se agrupan los estados del encoder (estado oculto y de celda) para pasarlos al decoder como estado inicial
encoder_states = [state_h, state_c]

# Entrada del decodificador (decoder): secuencia de longitud ANSWER_LEN codificada one-hot
decoder_inputs = Input(shape=(ANSWER_LEN, NUM_CHARS))

# LSTM del decoder: produce una salida en cada paso temporal (return_sequences=True),
# inicializa su estado con el estado final del encoder (encoder_states)
decoder_lstm = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Capa densa final: aplica softmax en cada paso para obtener una distribución sobre los caracteres posibles
decoder_dense = Dense(NUM_CHARS, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Se define el modelo completo que toma como entrada las secuencias del encoder y del decoder
# y devuelve como salida la secuencia de caracteres generada por el decoder
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compilación del modelo: se usa entropía cruzada categórica (por carácter) y el optimizador Adam
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [11]:
from tensorflow.keras.utils import plot_model
model.summary()
#plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 7, 12)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 4, 12)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 300),     │    375,600 │ input_layer[0][0] │
│                     │ (None, 300),      │            │                   │
│                     │ (None, 300)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 4, 300),  │    375,600 │ input_layer_1[0]… │
│                     │ (None, 300),      │            │ lstm[0][1],       │
│                     │ (None, 300)]      │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 4, 12)     │      3,612 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 754,812 (2.88 MB)

 Trainable params: 754,812 (2.88 MB)

 Non-trainable params: 0 (0.00 B)

# Aplicamos el Teacher Forcing


In [12]:

# Creamos una copia vacía con la misma forma que y_train (salidas esperadas one-hot).
# Esta matriz será usada como entrada del decoder durante el entrenamiento.
decoder_input_data = np.zeros_like(y_train)

# Aplicamos "teacher forcing":
# Desplazamos las salidas correctas un paso hacia la derecha para que el modelo reciba,
# en cada paso, el carácter correcto anterior como entrada.
decoder_input_data[:, 1:, :] = y_train[:, :-1, :]

# Insertamos un token de inicio (' ') en la primera posición de cada secuencia de entrada del decoder.
# Este token le indica al modelo que debe comenzar a generar la salida desde ahí.
decoder_input_data[:, 0, char_to_index[' ']] = 1  # Start token (puede reemplazarse por otro caracter especial)

# Entrenamiento
model.fit([x_train, decoder_input_data], y_train,
          batch_size=64, epochs=5, validation_split=0.2)






Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 88s 67ms/step - accuracy: 0.3624 - loss: 1.7354 - val_accuracy: 0.4683 - val_loss: 1.3556
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 86s 69ms/step - accuracy: 0.5339 - loss: 1.2199 - val_accuracy: 0.6745 - val_loss: 0.8752
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 141s 68ms/step - accuracy: 0.6984 - loss: 0.8025 - val_accuracy: 0.7443 - val_loss: 0.6528
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 144s 70ms/step - accuracy: 0.7686 - loss: 0.5969 - val_accuracy: 0.9050 - val_loss: 0.2888
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 140s 68ms/step - accuracy: 0.9165 - loss: 0.2407 - val_accuracy: 0.9671 - val_loss: 0.1093


# Modelos para Inferencia


In [13]:

# MODELOS PARA INFERENCIA

# Modelo encoder: toma la secuencia de entrada y devuelve los estados internos finales
encoder_model = Model(encoder_inputs, encoder_states)

# Inputs para los estados previos del decoder (serán alimentados en cada paso durante la generación)
decoder_state_input_h = Input(shape=(HIDDEN_SIZE,))
decoder_state_input_c = Input(shape=(HIDDEN_SIZE,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Input para el carácter anterior (one-hot, longitud 1) que alimenta el decoder en cada paso
decoder_input_single = Input(shape=(1, NUM_CHARS))

# LSTM que procesa un paso de decodificación, usando los estados anteriores
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_input_single, initial_state=decoder_states_inputs)

# Nuevos estados actualizados después de procesar este paso
decoder_states = [state_h, state_c]

# Proyección al espacio de caracteres (probabilidades softmax del siguiente carácter)
decoder_outputs = decoder_dense(decoder_outputs)

# Modelo decoder: recibe el carácter anterior y los estados, devuelve predicción y nuevos estados
decoder_model = Model(
    [decoder_input_single] + decoder_states_inputs,  # entradas
    [decoder_outputs] + decoder_states)              # salidas


In [14]:
encoder_model.summary()
#plot_model(encoder_model, to_file='encoder_model.png', show_shapes=True, show_layer_names=True)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 7, 12)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ [(None, 300), (None,   │       375,600 │
│                                 │ 300), (None, 300)]     │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 375,600 (1.43 MB)

 Trainable params: 375,600 (1.43 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
decoder_model.summary()
#plot_model(decoder_model, to_file='decoder_model.png', show_shapes=True, show_layer_names=True)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 1, 12)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 1, 300),  │    375,600 │ input_layer_4[0]… │
│                     │ (None, 300),      │            │ input_layer_2[0]… │
│                     │ (None, 300)]      │            │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1, 12)     │      3,612 │ lstm_1[1][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 379,212 (1.45 MB)

 Trainable params: 379,212 (1.45 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:



def decode_sequence(input_seq):
    # Ejecuta el encoder para obtener los estados iniciales del decoder
    states_value = encoder_model.predict(input_seq)

    # Primer input del decoder: vector one-hot del token de inicio (espacio)
    target_seq = np.zeros((1, 1, NUM_CHARS))
    target_seq[0, 0, char_to_index[' ']] = 1  # ' ' como start token - >[0,0,0,0,0,1,,0,0]

    decoded = ''  # Acumula la salida generada

    for _ in range(ANSWER_LEN):  # Hasta longitud máxima de respuesta
        # Ejecuta un paso del decoder con el input actual y estados anteriores
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Selecciona el carácter con mayor probabilidad
        sampled_idx = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_char[sampled_idx]
        decoded += sampled_char

        # Si el carácter generado es espacio, interpretamos fin de secuencia
        if sampled_char == ' ':
            break

        # Prepara el siguiente input: carácter recién generado
        target_seq = np.zeros((1, 1, NUM_CHARS))
        target_seq[0, 0, sampled_idx] = 1

        # Actualiza los estados para el próximo paso del decoder
        states_value = [h, c]

    return decoded.strip()  # Devuelve la cadena generada, sin espacios extra


def solve_equation(eq):
  a, b = eq.split('+')
  return int(a) + int(b)






In [18]:
# Testeo del modelo

for i in range(10):
    input_str = questions[TRAINING_SIZE + i]
    input_vec = vectorize([input_str], MAXLEN)
    pred = decode_sequence(input_vec)
    mark = '✅' if  eval(pred+'+0') == eval(answers[TRAINING_SIZE + i].strip())else '❌'
    print(f'{input_str.strip()} = {pred} (real: {answers[TRAINING_SIZE + i].strip()}) {mark}')



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
526+136 = 662 (real: 662) ✅
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
418+971 = 1389 (real: 1389) ✅
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
741+938 = 1679 (real: 1679) ✅
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
946+1 = 947 (real: 947) ✅
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━

In [19]:
new_questions  = [ '1+1', '456+362', '1+17', '589+932', '890+110', '666+20 ', '1235+15']
for q in new_questions:
    input_vec = vectorize([q.ljust(MAXLEN)], MAXLEN)
    pred = decode_sequence(input_vec)
    target = solve_equation(q)
    mark = '✅' if  eval(pred+'+0') == target else '❌'
    print(f'{q} = {pred} (real: {target}) {mark}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1+1 = 2 (real: 2) ✅
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
456+362 = 818 (real: 818) ✅
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1+17 = 20 (real: 18) ❌
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
589+932 = 1521 (real: 1521) ✅
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
890+110 = 1000 (real: 1000) ✅
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/